In [1]:
import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker
from textgrad.loss import TextLoss

In [2]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

/Users/eugeniusms/Development/SKRIPSI/sevet/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")

loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""",
                              requires_grad=False,
                              role_description="system prompt")


In [4]:
optimizer = TextualGradientDescent([solution])

In [5]:
loss = TextLoss(loss_system_prompt, engine=engine)
loss_value = loss(solution) # Forward method in Loss Function
loss_value

Variable(value=The calculation of  b² - 4ac was incorrect: it should be (-7)² - 4 * 3 * 2 = 49 - 24 = 25, not 49 + 24 = 73.  Also, the division by 6 is missing in the final answers for x1 and x2.
, role=response from the language model, grads=set())

In [6]:
# Optimize
loss_value.backward()

optimizer.step()
print("Optimized Solution:", solution.value)

['Here is a conversation:\n\n<CONVERSATION><LM_SYSTEM_PROMPT> You will evaluate a solution to a math question. \nDo not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise. </LM_SYSTEM_PROMPT>\n\n<LM_INPUT> To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:\nx = (-b ± √(b^2 - 4ac)) / 2a\na = 3, b = -7, c = 2\nx = (7 ± √((-7)^2 + 4(3)(2))) / 6\nx = (7 ± √73) / 6\nThe solutions are:\nx1 = (7 + √73)\nx2 = (7 - √73) </LM_INPUT>\n\n<LM_OUTPUT> The calculation of  b² - 4ac was incorrect: it should be (-7)² - 4 * 3 * 2 = 49 - 24 = 25, not 49 + 24 = 73.  Also, the division by 6 is missing in the final answers for x1 and x2.\n </LM_OUTPUT>\n\n</CONVERSATION>\n\nThis conversation is potentially part of a larger system. The output is used as response from the language model\n\nHere is the feedback we got for solution to the math question in the conversation:\n\n<FEEDBACK>The language model evaluation points out two specific errors in th

In [7]:
verification_task_prompts = [
    # Perspective 1: Rule-based verifier (objective, procedural)
    """
    Evaluate the calculation step strictly based on mathematical correctness and procedural rules. 
    If the step violates any algebraic or logical principle, replace it with the corrected version of that step only. 
    Do not proceed to solve the full problem.
    """,

    # Perspective 2: Teaching assistant (didactic, pedagogical)
    """
    Review the calculation step from the perspective of a teaching assistant helping a student learn. 
    If there's an error or suboptimal explanation, provide a corrected version that would best aid the student's understanding. 
    Focus only on the step in question, without solving the full problem.
    """
]

verifier = TextualVerifierWithTracker(
    verifier_engine=engine, 
    use_cot_generation=True,
    use_step_breakdown=True,
    verification_task_prompts=verification_task_prompts,
    enable_logging=True
)

instance = Variable(f"initial_solution: {initial_solution}\nloss_value: {loss_value}",
                    requires_grad=False,
                    role_description="instance")

optimizer_instruction = Variable("""You will optimize $initial_solution based on $loss_value. Be super concise.""",
                                requires_grad=False,
                                role_description="optimizer prompt")

verified_result = verifier.verify(instance=instance,
                                  instruction=optimizer_instruction,
                                  calculation=solution)

print("VERIFIED RESULT:", verified_result.value)

INFO:textgrad:TextualVerifier: Start verification process...
INFO:textgrad:TextualVerifier: Ready to verify 8 calculation steps...
INFO:textgrad:TextualVerifier: Verifying step 1/8...
INFO:textgrad:TextualVerifier: Generating step 1 variant 1/2...
INFO:textgrad:TextualVerifier: Generating step 1 variant 2/2...
INFO:textgrad:TextualVerifier: Running majority voting for step 1...
INFO:textgrad:TextualVerifier: Verifying step 2/8...
INFO:textgrad:TextualVerifier: Generating step 2 variant 1/2...
INFO:textgrad:TextualVerifier: Generating step 2 variant 2/2...
INFO:textgrad:TextualVerifier: Running majority voting for step 2...
INFO:textgrad:TextualVerifier: Verifying step 3/8...
INFO:textgrad:TextualVerifier: Generating step 3 variant 1/2...
INFO:textgrad:TextualVerifier: Generating step 3 variant 2/2...
INFO:textgrad:TextualVerifier: Running majority voting for step 3...
INFO:textgrad:TextualVerifier: Verifying step 4/8...
INFO:textgrad:TextualVerifier: Generating step 4 variant 1/2...
IN

In [8]:
import json

tracker_data = verifier.get_tracker()

with open('tracker_results/optimizer_verification.json', 'w') as f:
    json.dump(tracker_data, f, indent=4, default=str)

In [9]:
tracker_data

{'setup': {'verifier_engine': <textgrad.engine.gemini.ChatGemini at 0x10e919c10>,
  'use_cot_generation': True,
  'use_step_breakdown': True,
  'verification_task_prompts': ['\n    Evaluate the calculation step strictly based on mathematical correctness and procedural rules. \n    If the step violates any algebraic or logical principle, replace it with the corrected version of that step only. \n    Do not proceed to solve the full problem.\n    ',
   "\n    Review the calculation step from the perspective of a teaching assistant helping a student learn. \n    If there's an error or suboptimal explanation, provide a corrected version that would best aid the student's understanding. \n    Focus only on the step in question, without solving the full problem.\n    "],
  'enable_logging': True},
 'variable': {'instance': Variable(value=initial_solution: To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
  x = (-b ± √(b^2 - 4ac)) / 2a
  a = 3, b = -7, c = 2
  x = (7 ± √((